# Create a 20 Questions Answer with Rigging

In [1]:
import jupyter_black

jupyter_black.load()

# Question Asker Agent

In [1]:
import rigging as rg

import ipywidgets as widgets
from IPython.display import display, clear_output


class AskerQuestion(rg.Model):
    question: str

In [40]:

questions = []
answers = []

def ask_next_question(questions, answers, verbose=False):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to ask questions.
            - These questions should try to quickly find the keyword.
            - You will recieve back only yes or no responses.
            - Note previous questions and their answers when creating your next question
            - Do not assume anything specific too soon. Ask broad questions that could divide all possible remaining things into an even split.
            - The keyword will be a person, place or thing.
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}
        
        Ask your question within the tag {AskerQuestion.xml_tags()}
    """
    asker = (
        rg.get_generator("gpt-4o")
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )

    question = asker.last.parse(AskerQuestion).question
    if verbose:
        print(f"=== Question {len(questions) + 1} ====")
        print(question)

    return question

In [3]:
question = ask_next_question(questions, answers)

=== Question 1 ====
Is the keyword a living thing?


# LLM Trying to Guess the Keyword "TOMATO"

In [4]:
answer = "no"
questions.append(question)
answers.append(answer)
question = ask_next_question(questions, answers)

=== Question 2 ====
Is the keyword a place?


In [26]:
answer = "no, its a red fruit used in sauses with spagetti"
questions.append(question)
answers.append(answer)
question = ask_next_question(questions, answers)

=== Question 24 ====
Is the keyword a tomato?


# Create our Answerer Agent

In [37]:
class YesNoAnswer(rg.Model):
    "Yes/No answer answer with coercion"
    answer: str


def get_agent_answer(keyword, question):
    
    system_prompt = f"""
        You are playing the game 20 questions. You will be asked a question an must answer yes or no.
        The keyword you are answering for is {keyword}
    """
    
    user_prompt = f"""
        The question is <question>{question}</question>
    
        Your response should be given between the tags {YesNoAnswer.xml_tags()}
    
        It should only be 'yes' or 'no'
        
        """
    
    answerer = (
        rg.get_generator("gpt-4o")
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )
    
    answer = answerer.last.parse(YesNoAnswer).answer
    return answer

In [39]:
keyword = 'Tom Hanks'
question = 'Is he a handsome lad?'
get_agent_answer(keyword, question)

'yes'

# Guesser Agent

In [84]:
class Guess(rg.Model):
    "20 Questions guess"
    guess: str


def make_guess(questions : list, answers : list, guesses : list, verbose=False):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to guess the keyword.
            - Note previous questions and their answers when creating your guess
            - The keyword will be a person, place or thing.
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}

        Previous Guesses are, do not repeat these:
            {','.join(guesses)}
        
        You will now guess for the 20 question game.
          - Be specific.
          - Guess only a single thing.
        
          
        Put the word for your guess in the following tag {Guess.xml_tags()}
    """
    asker = (
        rg.get_generator("gpt-4o")
        .chat(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ]
        )
        .run()
    )

    guess = asker.last.parse(Guess).guess
    if verbose:
        print(f"=== Guess {len(questions) + 1} ====")
        print(guess)

    return guess

# Lets let the Agents fight it out!!!

In [85]:
import string

def keyword_guessed(guess: str, keyword: str) -> bool:
    def normalize(s: str) -> str:
      t = str.maketrans("", "", string.punctuation)
      return s.lower().replace("the", "").replace(" ", "").translate(t)

    if normalize(guess) == normalize(keyword):
      return True
    # for s in alts:
    #   if normalize(s) == normalize(guess):
    #     return True

    return False


In [87]:
keyword = 'Tomato'
questions = []
answers = []
guesses = []
max_questions = 20
for i in range(max_questions):
    question = ask_next_question(questions, answers)
    answer = get_agent_answer(keyword, question)
    print('=' * 20)
    print(f'Round {i}')
    print(f'[Question]: {question}')
    print(f'[Answer]:   {answer}')
    questions.append(question)
    answers.append(answer)
    guess = make_guess(questions, answers, guesses)
    guesses.append(guess)
    print(f'[Guess]: {guess}')
    if keyword_guessed(guess, keyword):
        print('FOUND IT!!!! BIZNATCH!!!!!!')
        break

Round 0
[Question]: Is the keyword a person?
[Answer]:   no
[Guess]: Mount Everest
Round 1
[Question]: Is the keyword a place?
[Answer]:   no
[Guess]: Golden Gate Bridge
Round 2
[Question]: Is the keyword a physical object?
[Answer]:   yes
[Guess]: Eiffel Tower
Round 3
[Question]: Is the keyword something that is primarily used indoors?
[Answer]:   no
[Guess]: Statue of Liberty
Round 4
[Question]: Is the keyword something that is found in nature?
[Answer]:   yes
[Guess]: Grand Canyon
Round 5
[Question]: Is the keyword something that is living?
[Answer]:   no
[Guess]: The Great Barrier Reef
Round 6
[Question]: Is the keyword a type of mineral or rock?
[Answer]:   no
[Guess]: Niagara Falls
Round 7
[Question]: Is the keyword a type of plant?
[Answer]:   yes
[Guess]: Baobab Tree
Round 8
[Question]: Is the keyword a type of tree?
[Answer]:   no
[Guess]: Cactus
Round 9
[Question]: Is the keyword something typically used in cooking?
[Answer]:   yes
[Guess]: Rosemary
Round 10
[Question]: Is th

# Can GPT-4o Guess Pizza in 20 Questions?!

In [89]:
keyword = 'Pizza'
questions = []
answers = []
guesses = []
max_questions = 20

for i in range(max_questions):
    question = ask_next_question(questions, answers)
    answer = get_agent_answer(keyword, question)
    print('=' * 20)
    print(f'Round {i+1}')
    print(f'[Question]: {question}')
    print(f'[Answer]:   {answer}')
    questions.append(question)
    answers.append(answer)
    guess = make_guess(questions, answers, guesses)
    guesses.append(guess)
    print(f'[Guess]: {guess}')
    if keyword_guessed(guess, keyword):
        print('FOUND IT!!!! BIZNATCH!!!!!!')
        break

Round 1
[Question]: Is the keyword a person?
[Answer]:   no
[Guess]: Mount Everest
Round 2
[Question]: Is the keyword a place?
[Answer]:   no
[Guess]: International Space Station
Round 3
[Question]: Is the keyword something tangible?
[Answer]:   yes
[Guess]: The Great Wall of China
Round 4
[Question]: Is the keyword something used in daily life?
[Answer]:   yes
[Guess]: Smartphone
Round 5
[Question]: Is the keyword an electronic device?
[Answer]:   no
[Guess]: Toothbrush
Round 6
[Question]: Is the keyword something typically found in a household?
[Answer]:   yes
[Guess]: Refrigerator
Round 7
[Question]: Is the keyword something primarily used in the kitchen?
[Answer]:   no
[Guess]: Chair
Round 8
[Question]: Is the keyword something used for cleaning?
[Answer]:   no
[Guess]: Television
Round 9
[Question]: Is the keyword furniture?
[Answer]:   no
[Guess]: Bed
Round 10
[Question]: Is the keyword something related to personal care?
[Answer]:   no
[Guess]: Book
Round 11
[Question]: Is the k